---
title: "Logic & Inference - Poole & Mackworth"
format: html
page-layout: full
code-line-numbers: true
code-block-border: true
toc: true
toc-location: left
number-sections: true
jupyter: python3
---

# Representing Knowledge Bases

- A clause consists of a head (an atom) and a body. 
- A body is represented as a list of atoms. 
- Atoms are represented as strings.

In [8]:
class Clause(object):
    """A definite clause"""

    def __init__(self,head,body=[]):
        """clause with atom head and list of atoms body"""
        self.head=head
        self.body = body

    def __repr__(self):
        """returns the string representation of a clause.
        """
        if self.body:
            return self.head + " <- " + " & ".join(str(a) for a in self.body) + "\n"
        else:
            return self.head + "."

In [9]:
Clause('x')

x.

In [10]:
Clause('a', ['b', 'c'])

a <- b & c

- An askable atom can be asked of the user.
- The user can respond in English or French or just with a “y”.

In [11]:
class Askable(object):
    """An askable atom"""

    def __init__(self,atom):
        """clause with atom head and list of atoms body"""
        self.atom=atom

    def __str__(self):
        """returns the string representation of a clause."""
        return "askable " + self.atom + "."

In [12]:
def yes(ans):
    """returns true if the answer is yes in some form"""
    return ans.lower() in ['yes', 'yes.', 'oui', 'oui.', 'y', 'y.']    # bilingual

 - A knowledge base is a list of clauses and askables.
   -  In order to make top-down inference faster, this creates a dictionary that maps each atoms into the set of clauses with that atom in the head.

In [13]:
class Displayable(object):
    """Class that uses 'display'.
    The amount of detail is controlled by max_display_level
    """
    max_display_level = 1   # can be overridden in subclasses or instances

    def display(self,level,*args,**nargs):
        """print the arguments if level is less than or equal to the
        current max_display_level.
        level is an integer.
        the other arguments are whatever arguments print can take.
        """
        if level <= self.max_display_level:
            print(*args, **nargs)  ##if error you are using Python2 not Python3
        

In [14]:
class KB(Displayable):
    """A knowledge base consists of a set of clauses.
    This also creates a dictionary to give fast access to the clauses with an atom in head.
    """
    def __init__(self, statements=[]):
        self.statements = statements
        self.clauses = [c for c in statements if isinstance(c, Clause)]
        self.askables = [c.atom for c in statements if isinstance(c, Askable)]
        self.atom_to_clauses = {}  # dictionary giving clauses with atom as head
        for c in self.clauses:
            self.add_clause(c)

    def add_clause(self, c):
        if c not in self.statements:
            self.statements.append(c)
            
        if c.head in self.atom_to_clauses:
            self.atom_to_clauses[c.head].add(c)
        else:
            self.atom_to_clauses[c.head] = {c}

    def clauses_for_atom(self,a):
        """returns set of clauses with atom a as the head"""
        if a in self.atom_to_clauses:
            return  self.atom_to_clauses[a]
        else:
            return set()

    def __str__(self):
        """returns a string representation of this knowledge base.
        """
        return '\n'.join([str(c) for c in self.statements])

In [15]:
triv_KB = KB([
    Clause('i_am', ['i_think']),
    Clause('i_think'),
    Clause('i_smell', ['i_exist'])
    ])

print(triv_KB)

i_am <- i_think

i_think.
i_smell <- i_exist



![](https://artint.info/3e/html/x42.png)

In [16]:
elect_KB = KB([
    Clause('light_l1'),
    Clause('light_l2'),
    Clause('ok_l1'),
    Clause('ok_l2'),
    Clause('ok_cb1'),
    Clause('ok_cb2'),
    Clause('live_outside'),
    Clause('live_l1', ['live_w0']),
    Clause('live_w0', ['up_s2','live_w1']),
    Clause('live_w0', ['down_s2','live_w2']),
    Clause('live_w1', ['up_s1', 'live_w3']),
    Clause('live_w2', ['down_s1','live_w3' ]),
    Clause('live_l2', ['live_w4']),
    Clause('live_w4', ['up_s3','live_w3' ]),
    Clause('live_p_1', ['live_w3']),
    Clause('live_w3', ['live_w5', 'ok_cb1']),
    Clause('live_p_2', ['live_w6']),
    Clause('live_w6', ['live_w5', 'ok_cb2']),
    Clause('live_w5', ['live_outside']),
    Clause('lit_l1', ['light_l1', 'live_l1', 'ok_l1']),
    Clause('lit_l2', ['light_l2', 'live_l2', 'ok_l2']),
    Askable('up_s1'),
    Askable('down_s1'),
    Askable('up_s2'),
    Askable('down_s2'),
    Askable('up_s3'),
    Askable('down_s2')
    ])

print(elect_KB)

light_l1.
light_l2.
ok_l1.
ok_l2.
ok_cb1.
ok_cb2.
live_outside.
live_l1 <- live_w0

live_w0 <- up_s2 & live_w1

live_w0 <- down_s2 & live_w2

live_w1 <- up_s1 & live_w3

live_w2 <- down_s1 & live_w3

live_l2 <- live_w4

live_w4 <- up_s3 & live_w3

live_p_1 <- live_w3

live_w3 <- live_w5 & ok_cb1

live_p_2 <- live_w6

live_w6 <- live_w5 & ok_cb2

live_w5 <- live_outside

lit_l1 <- light_l1 & live_l1 & ok_l1

lit_l2 <- light_l2 & live_l2 & ok_l2

askable up_s1.
askable down_s1.
askable up_s2.
askable down_s2.
askable up_s3.
askable down_s2.


# Bottom-up Proofs

- Derive all logical consequences of a knowledge base
- Rule of inference called **modus ponens**
  - If h &larr; a~1~ &and; a~2~ &and; ... a~m~ is a definite clause in the knowledge base, and each a~i~ has been derived, then *h* can be derived.

In [17]:
def ask_askables(kb):
    return {at for at in kb.askables if yes(input("Is " + at + " true? "))}

In [18]:
def fixed_point(kb):
    """Returns the fixed point of knowledge base kb.
    """
    fp = ask_askables(kb)
    added = True
    while added:
        added = False   # added is true when an atom was added to fp this iteration
        for c in kb.clauses:
            if c.head not in fp and all(b in fp for b in c.body):
                fp.add(c.head)
                added = True
                kb.display(2,c.head,"added to fp due to clause",c)
    return fp


In [19]:
print(triv_KB)

i_am <- i_think

i_think.
i_smell <- i_exist



In [20]:
def test1(kb=triv_KB, fixedpt = {'i_am','i_think'}):
    fp = fixed_point(kb)
    assert fp == fixedpt, f"kb gave result {fp}"
    print("Passed unit test")

In [21]:
test1()

Passed unit test


In [22]:
triv_KB.max_display_level = 3

fixed_point(triv_KB)

i_think added to fp due to clause i_think.
i_am added to fp due to clause i_am <- i_think



{'i_am', 'i_think'}

In [23]:
print(elect_KB)

light_l1.
light_l2.
ok_l1.
ok_l2.
ok_cb1.
ok_cb2.
live_outside.
live_l1 <- live_w0

live_w0 <- up_s2 & live_w1

live_w0 <- down_s2 & live_w2

live_w1 <- up_s1 & live_w3

live_w2 <- down_s1 & live_w3

live_l2 <- live_w4

live_w4 <- up_s3 & live_w3

live_p_1 <- live_w3

live_w3 <- live_w5 & ok_cb1

live_p_2 <- live_w6

live_w6 <- live_w5 & ok_cb2

live_w5 <- live_outside

lit_l1 <- light_l1 & live_l1 & ok_l1

lit_l2 <- light_l2 & live_l2 & ok_l2

askable up_s1.
askable down_s1.
askable up_s2.
askable down_s2.
askable up_s3.
askable down_s2.


In [24]:
elect_KB.max_display_level = 3

fixed_point(elect_KB)

KeyboardInterrupt: Interrupted by user

# Top-down Proofs (with askables)

In [25]:
def prove(kb, ans_body, indent=""):
    """returns True if kb |- ans_body
    ans_body is a list of atoms to be proved
    """
    kb.display(2,indent,'yes <-',' & '.join(ans_body))
    if ans_body:
        selected = ans_body[0]   # select first atom from ans_body
        if selected in kb.askables:
            return (yes(input("Is "+selected+" true? "))
                    and  prove(kb,ans_body[1:],indent+"    "))
        else:
            return any(prove(kb,cl.body+ans_body[1:],indent+"    ")
                       for cl in kb.clauses_for_atom(selected))
    else:
        return True    # empty body is true

In [26]:
print(triv_KB)

i_am <- i_think

i_think.
i_smell <- i_exist



In [27]:
prove(triv_KB,['i_am'])

 yes <- i_am
     yes <- i_think
         yes <- 


True

In [28]:
prove(triv_KB, ['i_smell'])

 yes <- i_smell
     yes <- i_exist


False

In [29]:
print(elect_KB)

light_l1.
light_l2.
ok_l1.
ok_l2.
ok_cb1.
ok_cb2.
live_outside.
live_l1 <- live_w0

live_w0 <- up_s2 & live_w1

live_w0 <- down_s2 & live_w2

live_w1 <- up_s1 & live_w3

live_w2 <- down_s1 & live_w3

live_l2 <- live_w4

live_w4 <- up_s3 & live_w3

live_p_1 <- live_w3

live_w3 <- live_w5 & ok_cb1

live_p_2 <- live_w6

live_w6 <- live_w5 & ok_cb2

live_w5 <- live_outside

lit_l1 <- light_l1 & live_l1 & ok_l1

lit_l2 <- light_l2 & live_l2 & ok_l2

askable up_s1.
askable down_s1.
askable up_s2.
askable down_s2.
askable up_s3.
askable down_s2.


In [30]:
elect_KB.max_display_level = 3

prove(elect_KB, ['live_w6'])

 yes <- live_w6
     yes <- live_w5 & ok_cb2
         yes <- live_outside & ok_cb2
             yes <- ok_cb2
                 yes <- 


True

In [24]:
prove(elect_KB, ['live_w1'])

 yes <- live_w1
     yes <- up_s1 & live_w3


Is up_s1 true?  n


False

In [25]:
prove(elect_KB, ['lit_l1'])

 yes <- lit_l1
     yes <- light_l1 & live_l1 & ok_l1
         yes <- live_l1 & ok_l1
             yes <- live_w0 & ok_l1
                 yes <- up_s2 & live_w1 & ok_l1


Is up_s2 true?  n


                 yes <- down_s2 & live_w2 & ok_l1


Is down_s2 true?  y


                     yes <- live_w2 & ok_l1
                         yes <- down_s1 & live_w3 & ok_l1


Is down_s1 true?  y


                             yes <- live_w3 & ok_l1
                                 yes <- live_w5 & ok_cb1 & ok_l1
                                     yes <- live_outside & ok_cb1 & ok_l1
                                         yes <- ok_cb1 & ok_l1
                                             yes <- ok_l1
                                                 yes <- 


True

# Explanation

 - The top-down procedure is modified to build a proof tree than can be traversed for explanation and debugging.
 - prove_atom(kb,atom) returns a proof for atom from a knowledge base kb, where a proof is a pair of the atom and the proofs for the elements of the body of the clause used to prove the atom.
 - prove_body(kb,body) returns a list of proofs for list body from a knowledge base, kb

In [26]:
def prove_atom(kb, atom, indent=""):
    """returns a pair (atom,proofs) where proofs is the list of proofs 
       of the elements of a body of a clause used to prove atom.
    """
    kb.display(2,indent,'proving',atom)
    if atom in kb.askables:
        if yes(input("Is "+atom+" true? ")):
            return (atom,"answered")
        else:
            return "fail"
    else:
        for cl in kb.clauses_for_atom(atom):
            kb.display(2,indent,"trying",atom,'<-',' & '.join(cl.body))
            pr_body = prove_body(kb, cl.body, indent)
            if pr_body != "fail":
                return (atom, pr_body)
        return "fail"

In [27]:
def prove_body(kb, ans_body, indent=""):
    """returns proof tree if kb |- ans_body or "fail" if there is no proof 
    ans_body is a list of atoms in a body to be proved
    """
    proofs = []
    for atom in ans_body:
        proof_at = prove_atom(kb, atom, indent+"  ")
        if proof_at == "fail":
            return "fail"  # fail if any proof fails
        else:
            proofs.append(proof_at)
    return proofs

In [28]:
triv_KB.max_display_level = 3

prove_atom(triv_KB,'i_am')

 proving i_am
 trying i_am <- i_think
   proving i_think
   trying i_think <- 


('i_am', [('i_think', [])])

In [29]:
prove_atom(triv_KB,'i_smell')

 proving i_smell
 trying i_smell <- i_exist
   proving i_exist


'fail'

In [30]:
elect_KB.max_display_level=3

prove_atom(elect_KB, 'live_w6')

 proving live_w6
 trying live_w6 <- live_w5 & ok_cb2
   proving live_w5
   trying live_w5 <- live_outside
     proving live_outside
     trying live_outside <- 
   proving ok_cb2
   trying ok_cb2 <- 


('live_w6', [('live_w5', [('live_outside', [])]), ('ok_cb2', [])])

In [31]:
prove_atom(elect_KB, 'live_w1')

 proving live_w1
 trying live_w1 <- up_s1 & live_w3
   proving up_s1


Is up_s1 true?  n


'fail'

In [32]:
prove_atom(elect_KB, 'lit_l1')

 proving lit_l1
 trying lit_l1 <- light_l1 & live_l1 & ok_l1
   proving light_l1
   trying light_l1 <- 
   proving live_l1
   trying live_l1 <- live_w0
     proving live_w0
     trying live_w0 <- up_s2 & live_w1
       proving up_s2


Is up_s2 true?  n


     trying live_w0 <- down_s2 & live_w2
       proving down_s2


Is down_s2 true?  y


       proving live_w2
       trying live_w2 <- down_s1 & live_w3
         proving down_s1


Is down_s1 true?  y


         proving live_w3
         trying live_w3 <- live_w5 & ok_cb1
           proving live_w5
           trying live_w5 <- live_outside
             proving live_outside
             trying live_outside <- 
           proving ok_cb1
           trying ok_cb1 <- 
   proving ok_l1
   trying ok_l1 <- 


('lit_l1',
 [('light_l1', []),
  ('live_l1',
   [('live_w0',
     [('down_s2', 'answered'),
      ('live_w2',
       [('down_s1', 'answered'),
        ('live_w3',
         [('live_w5', [('live_outside', [])]), ('ok_cb1', [])])])])]),
  ('ok_l1', [])])

# Negation as Failure

- The negation of an atom 'a' is written as Not(a) in a body.

In [33]:
class Not(object):
    def __init__(self, atom):
        self.theatom = atom

    def atom(self):
        return self.theatom

    def __repr__(self):
        return f"Not({self.theatom})"


- Prove with negation-as-failure (prove_naf) is like prove, but with the extra case to cover Not:

In [34]:
def prove_naf(kb, ans_body, indent=""):
    """ prove with negation-as-failure and askables
    returns True if kb |- ans_body
    ans_body is a list of atoms to be proved
    """
    kb.display(2,indent,'yes <-',' & '.join(str(e) for e in ans_body))
    if ans_body:
        selected = ans_body[0]   # select first atom from ans_body
        if isinstance(selected, Not):
            kb.display(2,indent,f"proving {selected.atom()}")
            if prove_naf(kb, [selected.atom()], indent):
                kb.display(2,indent,f"{selected.atom()} succeeded so Not({selected.atom()}) fails")
                return False
            else:
                kb.display(2,indent,f"{selected.atom()} fails so Not({selected.atom()}) succeeds")
                return prove_naf(kb, ans_body[1:],indent+"    ")
                
        if selected in kb.askables:
            return (yes(input("Is "+selected+" true? "))
                    and  prove_naf(kb,ans_body[1:],indent+"    "))
        else:
            return any(prove_naf(kb,cl.body+ans_body[1:],indent+"    ")
                       for cl in kb.clauses_for_atom(selected))
    else:
        return True    # empty body is true


In [35]:
triv_KB_naf = KB([
    Clause('i_am', ['i_think']),
    Clause('i_think'),
    Clause('i_smell', ['i_am', Not('dead')]),
    Clause('i_bad', ['i_am', Not('i_think')])
    ])

print(triv_KB_naf)

i_am <- i_think

i_think.
i_smell <- i_am & Not(dead)

i_bad <- i_am & Not(i_think)



In [36]:
triv_KB_naf.max_display_level = 3

In [37]:
prove_naf(triv_KB_naf,['i_smell'])

 yes <- i_smell
     yes <- i_am & Not(dead)
         yes <- i_think & Not(dead)
             yes <- Not(dead)
             proving dead
             yes <- dead
             dead fails so Not(dead) succeeds
                 yes <- 


True

In [38]:
beach_KB = KB([
   Clause('away_from_beach', [Not('on_beach')]),
   Clause('beach_access', ['on_beach', Not('ab_beach_access')]),
   Clause('swim_at_beach', ['beach_access', Not('ab_swim_at_beach')]),
   Clause('ab_swim_at_beach', ['enclosed_bay', 'big_city', Not('ab_no_swimming_near_city')]),
   Clause('ab_no_swimming_near_city', ['in_BC', Not('ab_BC_beaches')])
    ])

print(beach_KB)

away_from_beach <- Not(on_beach)

beach_access <- on_beach & Not(ab_beach_access)

swim_at_beach <- beach_access & Not(ab_swim_at_beach)

ab_swim_at_beach <- enclosed_bay & big_city & Not(ab_no_swimming_near_city)

ab_no_swimming_near_city <- in_BC & Not(ab_BC_beaches)



In [39]:
beach_KB.max_display_level = 4

In [40]:
prove_naf(beach_KB, ['away_from_beach'])

 yes <- away_from_beach
     yes <- Not(on_beach)
     proving on_beach
     yes <- on_beach
     on_beach fails so Not(on_beach) succeeds
         yes <- 


True

In [41]:
prove_naf(beach_KB, ['beach_access'])

 yes <- beach_access
     yes <- on_beach & Not(ab_beach_access)


False

In [42]:
beach_KB.add_clause(Clause('on_beach', []))

print(beach_KB)

away_from_beach <- Not(on_beach)

beach_access <- on_beach & Not(ab_beach_access)

swim_at_beach <- beach_access & Not(ab_swim_at_beach)

ab_swim_at_beach <- enclosed_bay & big_city & Not(ab_no_swimming_near_city)

ab_no_swimming_near_city <- in_BC & Not(ab_BC_beaches)

on_beach.


In [43]:
prove_naf(beach_KB, ['away_from_beach'])

 yes <- away_from_beach
     yes <- Not(on_beach)
     proving on_beach
     yes <- on_beach
         yes <- 
     on_beach succeeded so Not(on_beach) fails


False

In [44]:
prove_naf(beach_KB, ['swim_at_beach'])

 yes <- swim_at_beach
     yes <- beach_access & Not(ab_swim_at_beach)
         yes <- on_beach & Not(ab_beach_access) & Not(ab_swim_at_beach)
             yes <- Not(ab_beach_access) & Not(ab_swim_at_beach)
             proving ab_beach_access
             yes <- ab_beach_access
             ab_beach_access fails so Not(ab_beach_access) succeeds
                 yes <- Not(ab_swim_at_beach)
                 proving ab_swim_at_beach
                 yes <- ab_swim_at_beach
                     yes <- enclosed_bay & big_city & Not(ab_no_swimming_near_city)
                 ab_swim_at_beach fails so Not(ab_swim_at_beach) succeeds
                     yes <- 


True

In [45]:
beach_KB.add_clause(Clause('enclosed_bay',[]))

print(beach_KB)

away_from_beach <- Not(on_beach)

beach_access <- on_beach & Not(ab_beach_access)

swim_at_beach <- beach_access & Not(ab_swim_at_beach)

ab_swim_at_beach <- enclosed_bay & big_city & Not(ab_no_swimming_near_city)

ab_no_swimming_near_city <- in_BC & Not(ab_BC_beaches)

on_beach.
enclosed_bay.


In [46]:
prove_naf(beach_KB, ['swim_at_beach'])

 yes <- swim_at_beach
     yes <- beach_access & Not(ab_swim_at_beach)
         yes <- on_beach & Not(ab_beach_access) & Not(ab_swim_at_beach)
             yes <- Not(ab_beach_access) & Not(ab_swim_at_beach)
             proving ab_beach_access
             yes <- ab_beach_access
             ab_beach_access fails so Not(ab_beach_access) succeeds
                 yes <- Not(ab_swim_at_beach)
                 proving ab_swim_at_beach
                 yes <- ab_swim_at_beach
                     yes <- enclosed_bay & big_city & Not(ab_no_swimming_near_city)
                         yes <- big_city & Not(ab_no_swimming_near_city)
                 ab_swim_at_beach fails so Not(ab_swim_at_beach) succeeds
                     yes <- 


True

In [47]:
beach_KB.add_clause(Clause('big_city',[]))

print(beach_KB)

away_from_beach <- Not(on_beach)

beach_access <- on_beach & Not(ab_beach_access)

swim_at_beach <- beach_access & Not(ab_swim_at_beach)

ab_swim_at_beach <- enclosed_bay & big_city & Not(ab_no_swimming_near_city)

ab_no_swimming_near_city <- in_BC & Not(ab_BC_beaches)

on_beach.
enclosed_bay.
big_city.


In [48]:
prove_naf(beach_KB, ['swim_at_beach'])

 yes <- swim_at_beach
     yes <- beach_access & Not(ab_swim_at_beach)
         yes <- on_beach & Not(ab_beach_access) & Not(ab_swim_at_beach)
             yes <- Not(ab_beach_access) & Not(ab_swim_at_beach)
             proving ab_beach_access
             yes <- ab_beach_access
             ab_beach_access fails so Not(ab_beach_access) succeeds
                 yes <- Not(ab_swim_at_beach)
                 proving ab_swim_at_beach
                 yes <- ab_swim_at_beach
                     yes <- enclosed_bay & big_city & Not(ab_no_swimming_near_city)
                         yes <- big_city & Not(ab_no_swimming_near_city)
                             yes <- Not(ab_no_swimming_near_city)
                             proving ab_no_swimming_near_city
                             yes <- ab_no_swimming_near_city
                                 yes <- in_BC & Not(ab_BC_beaches)
                             ab_no_swimming_near_city fails so Not(ab_no_swimming_near_city) succeeds
       

False

In [49]:
beach_KB.add_clause(Clause('in_BC',[]))

print(beach_KB)

away_from_beach <- Not(on_beach)

beach_access <- on_beach & Not(ab_beach_access)

swim_at_beach <- beach_access & Not(ab_swim_at_beach)

ab_swim_at_beach <- enclosed_bay & big_city & Not(ab_no_swimming_near_city)

ab_no_swimming_near_city <- in_BC & Not(ab_BC_beaches)

on_beach.
enclosed_bay.
big_city.
in_BC.


In [50]:
prove_naf(beach_KB, ['swim_at_beach'])

 yes <- swim_at_beach
     yes <- beach_access & Not(ab_swim_at_beach)
         yes <- on_beach & Not(ab_beach_access) & Not(ab_swim_at_beach)
             yes <- Not(ab_beach_access) & Not(ab_swim_at_beach)
             proving ab_beach_access
             yes <- ab_beach_access
             ab_beach_access fails so Not(ab_beach_access) succeeds
                 yes <- Not(ab_swim_at_beach)
                 proving ab_swim_at_beach
                 yes <- ab_swim_at_beach
                     yes <- enclosed_bay & big_city & Not(ab_no_swimming_near_city)
                         yes <- big_city & Not(ab_no_swimming_near_city)
                             yes <- Not(ab_no_swimming_near_city)
                             proving ab_no_swimming_near_city
                             yes <- ab_no_swimming_near_city
                                 yes <- in_BC & Not(ab_BC_beaches)
                                     yes <- Not(ab_BC_beaches)
                                     proving a

True

# Relations

In [51]:
class Var(Displayable):
    """A logical variable"""
    def __init__(self, name):
        """name"""
        self.name = name

    def __str__(self):
        return self.name
    __repr__ = __str__
    
    def __eq__(self, other):
        return isinstance(other,Var) and self.name == other.name
        
    def __hash__(self):
        return hash(self.name)


In [52]:
class Atom(object):
    """An atom"""
    def __init__(self, name, args):
        self.name = name
        self.args = args

    def __str__(self):
        return f"{self.name}({', '.join(str(a) for a in self.args)})"
    __repr__ = __str__

In [53]:
Func = Atom  # same syntax is used for function symbols

In [54]:
class FOL_Clause(Clause):
    next_index=0
    def __init__(self, head, *args, **nargs):
        if not isinstance(head, Atom):
            head = Atom(head)
        Clause.__init__(self, head, *args, **nargs)
        self.logical_variables = log_vars([self.head,self.body],set())

    def rename(self):
        """create a unique copy of the clause"""
        if self.logical_variables:
            sub = {v:Var(f"{v.name}_{FOL_Clause.next_index}") for v in self.logical_variables}
            FOL_Clause.next_index += 1
            return Clause(apply(self.head,sub),apply(self.body,sub))
        else:
           return self
        

In [55]:
def log_vars(exp, vs):
    """the union the logical variables in exp and the set vs"""
    if isinstance(exp,Var):
        return {exp}|vs
    elif isinstance(exp,Atom):
        return log_vars(exp.name, log_vars(exp.args, vs))
    elif isinstance(exp,(list,tuple)):
        for e in exp:
            vs = log_vars(e, vs)
    return vs


In [56]:
unifdisp = Var(None) # for display

In [57]:
def unify(t1,t2):
    e = [(t1,t2)]
    s = {}  # empty dictionary
    while e:
        (a,b) = e.pop()
        unifdisp.display(2,f"unifying{(a,b)}, e={e},s={s}")
        if a != b:
            if isinstance(a,Var):
                e = apply(e,{a:b}) 
                s = apply(s,{a:b})
                s[a]=b
            elif isinstance(b,Var):
                e = apply(e,{b:a}) 
                s = apply(s,{b:a})
                s[b]=a
            elif isinstance(a,Atom) and isinstance(b,Atom) and a.name==b.name and len(a.args)==len(b.args):
                e += zip(a.args,b.args)
            elif isinstance(a,(list,tuple)) and isinstance(b,(list,tuple)) and len(a)==len(b ):
                e += zip(a,b)
            else:
                return False
    return s

In [58]:
def apply(e,sub):
    """e is an expression
    sub is a {var:val} dictionary
    returns e with all occurrence of var replaces with val"""
    if isinstance(e,Var) and e in sub:
        return sub[e]
    if isinstance(e,Atom):
        return Atom(e.name, apply(e.args,sub))
    if isinstance(e,list):
        return [apply(a,sub) for a in e]
    if isinstance(e,tuple):
        return tuple(apply(a,sub) for a in e)
    if isinstance(e,dict):
        return {k:apply(v,sub) for (k,v) in e.items()}
    else:
        return e

In [59]:
unifdisp.max_display_level = 1   # show trace
e1 = Atom('p',[Var('X'),Var('Y'),Var('Y')])
e2 = Atom('p',['a',Var('Z'),'b'])
apply(e1,{Var('Y'):'b'})
unify(e1,e2)

{Y: 'b', Z: 'b', X: 'a'}

In [60]:
e3 = Atom('p',['a',Var('Y'),Var('Y')])
e4 = Atom('p',[Var('Z'),Var('Z'),'b'])
unify(e3,e4)

False

In [61]:
class FOL_KB(KB):
    """A first-order knowledge base. 
      only the indexing is changed to index on name of the head."""
        
    def add_clause(self, c):
        """Add clause c to clause dictionary"""
        if c.head.name in self.atom_to_clauses:
            self.atom_to_clauses[c.head.name].append(c)
        else:
            self.atom_to_clauses[c.head.name] = [c]

    def ask(self, query):
        """self is the current KB
        query is a list of atoms to be proved
        generates {variable:value} dictionary"""

        qvars = list(log_vars(query, set()))
        for ans in self.prove(qvars, query):
            yield {x:v for (x,v) in zip(qvars,ans)}

    def ask_all(self, query):
        """returns a list of all answers to the query given kb"""
        return list(self.ask(query))

    def ask_one(self, query):
        """returns an answer to the query given kb or None of there are no answers"""
        for ans in self.ask(query):
            return ans

    def prove(self, ans, ans_body, indent=""):
        """enumerates the proofs for ans_body
        ans_body is a list of atoms to be proved
        ans is the list of values of the query variables
        """
        self.display(2,indent,f"(yes({ans}) <-"," & ".join(str(a) for a in ans_body))
        if ans_body==[]:
            yield ans
        else:
            selected, remaining = self.select_atom(ans_body)
            if self.built_in(selected):
                yield from self.eval_built_in(ans, selected, remaining, indent)
            else:
                for chosen_clause in self.atom_to_clauses[selected.name]:
                    clause = chosen_clause.rename()  # rename variables
                    sub = unify(selected, clause.head)
                    if sub is not False:
                        self.display(3,indent,"KB.prove: selected=", selected, "clause=",clause,"sub=",sub) 
                        resans = apply(ans,sub)
                        new_ans_body = apply(clause.body+remaining, sub)
                        yield from self.prove(resans, new_ans_body, indent+"    ")

    def select_atom(self,lst):
        """given list of atoms, return (selected atom, remaining atoms)
        """
        return lst[0],lst[1:]

    def built_in(self,atom):
        return atom.name in ['lt','triple']

    def eval_built_in(self,ans, selected, remaining, indent):
        if selected.name == 'lt':  # less than
            [a1,a2] = selected.args
            if a1 < a2:
                yield from self.prove(ans, remaining, indent+"    ")
        if selected.name == 'triple':    # use triple store (AIFCA Ch 16)
            yield from self.eval_triple(ans, selected, remaining, indent)


In [62]:
A = Var('A')
W = Var('W')
X = Var('X')
Y = Var('Y')
Z = Var('Z')

def cons(h,t): 
    return Atom('cons',[h,t])
    
def append(a,b,c): 
    return Atom('append',[a,b,c])

app_KB = FOL_KB([
    FOL_Clause(append('nil',W,W)),
    FOL_Clause(append(cons(A,X), Y,cons(A,Z)),
                [append(X,Y,Z)])
    ])

F = Var('F')
lst = cons('l',cons('i',cons('s',cons('t','nil'))))

app_KB.max_display_level = 1 #show derivation
#ask_all(app_KB, [append(F,cons(A,'nil'), lst)])

app_KB.ask_all([append(F,cons(A,'nil'), lst)])

[{F: cons(l, cons(i, cons(s, nil))), A: 't'}]

In [63]:
app_KB.ask_all([append(X, Y, lst)])

[{Y: cons(l, cons(i, cons(s, cons(t, nil)))), X: 'nil'},
 {Y: cons(i, cons(s, cons(t, nil))), X: cons(l, nil)},
 {Y: cons(s, cons(t, nil)), X: cons(l, cons(i, nil))},
 {Y: cons(t, nil), X: cons(l, cons(i, cons(s, nil)))},
 {Y: 'nil', X: cons(l, cons(i, cons(s, cons(t, nil))))}]

In [64]:
L = Var('L')

app_KB.ask_all([append(lst, lst, L), append(X, cons('s',Y), L)])

[{Y: cons(t, cons(l, cons(i, cons(s, cons(t, nil))))),
  L: cons(l, cons(i, cons(s, cons(t, cons(l, cons(i, cons(s, cons(t, nil)))))))),
  X: cons(l, cons(i, nil))},
 {Y: cons(t, nil),
  L: cons(l, cons(i, cons(s, cons(t, cons(l, cons(i, cons(s, cons(t, nil)))))))),
  X: cons(l, cons(i, cons(s, cons(t, cons(l, cons(i, nil))))))}]